<a href="https://colab.research.google.com/github/rajatjpatel/Bazaar/blob/main/SimpleStock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import pandas as pd
import glob
import requests, zipfile, io

#def downloaddata(month,dateInString):
   
#months = ["JAN" , "FEB" , "MAR" , "APR" , "JUN" , "JUL" , "AUG" , "SEP" , "OCT" , "NOV" , "DEC"]
months = ["JAN"]
dates = range(1,31)
    
for month in months:
    for date in dates:
        if date < 10:
             dateInString = str(0)+str(date)
        else:
            dateInString = str(date) 
        url = "https://www1.nseindia.com/content/historical/EQUITIES/2020/"+month+"/cm"+dateInString+month+"2020bhav.csv.zip"
        r = requests.get(url)
        temp=r.status_code
        if(temp!=404):
                z = zipfile.ZipFile(io.BytesIO(r.content))        
                z.extractall("/content/Equities")
        else:
                pass

path =r'/content/Equities' 
#allExtractedFiles = glob.glob(path + "\*.csv")
allExtractedFiles = glob.glob(path + "/content/Equities/*.csv")
#print(allExtractedFiles)
list = []
for file in allExtractedFiles:
    df = pd.read_csv(file,index_col=None, header=0 , engine = 'python')
    list.append(df)
    frame = pd.concat(list, axis = 0, ignore_index = True)
    frame.to_csv('MasterFileEquity1.csv', index= False)
    symbol_company_data = pd.read_csv("NewScripts.csv")
    stock_data = pd.read_csv('MasterFileEquity1.csv')
    stock_data['STOCKNAME']= stock_data['SYMBOL'].map(symbol_company_data.drop_duplicates('NSE SYMBOL').set_index('NSE SYMBOL')['COMPANY NAME']).fillna("Not available")

    file_name = 'MasterFileEquity_Jan_2020.csv'
    stock_data.to_csv(file_name, encoding='utf-8')

In [ ]:
import pandas as pd
#stock_data = pd.read_csv("/content/Equities/cm30JAN2020bhav.csv")
stock_data = pd.read_csv("MasterFileEquity_2020.csv")
columns = stock_data.columns
equity_stock_data = stock_data[stock_data['SERIES'] == 'EQ']
equity_stock_data ['DATE'] = pd.to_datetime(equity_stock_data.TIMESTAMP, dayfirst=True)

filtered_equity_stock_data = equity_stock_data[equity_stock_data['CLOSE'] < 50]
filtered_equity_stock_data = filtered_equity_stock_data[filtered_equity_stock_data['DATE'] <  '2020-05-01']

difference = []
for data, df_symbols in filtered_equity_stock_data.groupby('SYMBOL'):   
    df_symbols.sort_values(by="DATE" , inplace=True)
    start=  df_symbols['CLOSE'].iloc[0] # first element 
    end =  df_symbols['CLOSE'].iloc[-1]      
    difference =  start-end
    percentage_difference = (difference / start )* 100
    if percentage_difference > 60:
        print(df_symbols['SYMBOL'].iloc[0])
    
    
#pd.concat(df_symbols['SYMBOL'] , difference)
